In [8]:
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

In [9]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")
print(arxiv.name)

arxiv


In [10]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [11]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia, description="Query Wikipedia articles")
print(wikipedia.name)

wikipedia


In [12]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

In [14]:
tavily.invoke("Provide me the recent AI news")

[{'title': 'Artificial Intelligence News - ScienceDaily',
  'url': 'https://www.sciencedaily.com/news/computers_math/artificial_intelligence/',
  'content': "Tuesday, November 12, 2024\n\nGiving Robots Superhuman Vision Using Radio Signals\n\nMonday, November 11, 2024\n\nAI Can Detect Serious Neurologic Changes in Babies in the NICU Using Video Data Alone\nRobot That Watched Surgery Videos Performs With Skill of Human Doctor, Researchers Report\n\nThursday, November 7, 2024\n\nRobot Learns How to Clean a Washbasin\n\nWednesday, November 6, 2024\n\nAI-Driven Mobile Robots Team Up to Tackle Chemical Synthesis\n\nThursday, October 31, 2024 [...] Using Robots in Nursing Homes Linked to Higher Employee Retention, Better Patient Care\n\nMonday, January 6, 2025\n\nAI Slashes Cost and Time for Chip Design, but That Is Not All\n\nThursday, January 2, 2025\n\nArtificial Intelligence: Algorithms Improve Medical Image Analysis\n\nFriday, December 27, 2024\n\nCrossing the Uncanny Valley: Breakthrou

In [15]:
tools=[arxiv,wikipedia,tavily]

In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="qwen-qwq-32b")

In [17]:
llm.invoke("What is AI")

AIMessage(content='\n<think>\nOkay, the user is asking, "What is AI?" Let me start by breaking down what they might need. They could be a beginner looking for a simple explanation or someone with some background wanting a deeper dive. I should cover the basics first.\n\nAI stands for Artificial Intelligence. I should define it in simple terms, maybe compare it to human intelligence. Then mention the key areas like machine learning, deep learning, and neural networks. Wait, but how detailed should I get? The user might not know those terms yet, so I need to explain them briefly.\n\nAlso, applications are important. Examples like Siri, self-driving cars, recommendation systems. That makes it relatable. Maybe touch on types of AI: narrow vs general AI. Oh, right, there\'s also the difference between weak and strong AI. Should clarify that.\n\nI should also mention how AI works, maybe talk about data processing and algorithms. Not too technical, but enough to give an idea. Maybe a sentence

In [18]:
llm_with_tools = llm.bind_tools(tools=tools)

In [19]:
llm_with_tools.invoke("What is AI? Provide me the recent AI news and some arxiv papers about it.")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_te1m', 'function': {'arguments': '{"query": "recent AI news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_8bye', 'function': {'arguments': '{"query": "AI"}', 'name': 'arxiv'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 347, 'prompt_tokens': 333, 'total_tokens': 680, 'completion_time': 0.860014714, 'prompt_time': 0.018920881, 'queue_time': 0.018809379, 'total_time': 0.878935595}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_512a3da6bb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1d56dd0f-a13d-40eb-94ff-c048da3fbe7d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent AI news'}, 'id': 'call_te1m', 'type': 'tool_call'}, {'name': 'arxiv', 'args': {'query': 'AI'}, 'id': 'call_8bye', 'type': 'tool_call'}], usage_metadata={'input_tokens': 333, 'output_tokens': 347, 'total_tokens': 680})

In [20]:
from typing_extensions import TypedDict
from langchain_core.messages import  AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages


In [21]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    

In [22]:
from IPython.display import display,Image
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition




In [31]:
def tool_calling_llm(state:State):
    return{"messages":[llm_with_tools.invoke(state["messages"])]}
    

builder = StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
)
builder.add_edge("tools",END)

graph = builder.compile()






In [32]:
messages=graph.invoke({"messages":"1706.03672"})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

1706.03672
================================== Ai Message ==================================
Tool Calls:
  arxiv (call_6amt)
 Call ID: call_6amt
  Args:
    query: 1706.03672
================================= Tool Message =================================
Name: arxiv

Published: 2017-06-12
Title: Determination of the NNLO low-energy constant $C_{93}$
Authors: Maarten Golterman, Kim Maltman, Santiago Peris
Summary: Experimental data from hadronic $\tau$ decays allow for a precision
determination of the slope of the $I=1$ vacuum polarization at zero momentum.
We use this information to provide a value for the next-to-next-to-leading
order (NNLO) low-energy constant $C_{93}$ in chiral perturbation theory. The
largest systematic error in this determination result


In [30]:
messages=graph.invoke({"messages":"Hi my name is Dhoni"})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Hi my name is Dhoni
================================== Ai Message ==================================

Hello Dhoni! Nice to meet you. How can I assist you today?
